In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import plotnine as p9
import statsmodels.formula.api as smf

import plotnine as p9
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
"""
Ticker  Fund Name                           Issuer                          AUM         Expense Ratio       3-Mo TR     Segment
MSTU	T-Rex 2X Long MSTR Daily Target ETF	Tuttle Capital Management LLC	$850.96M	1.05%	            -68.3%	    Leveraged Equity: U.S. Application Software
""";

In [ ]:
# tickers = ['MSTU', 'MSTR']
# ydata = yf.Tickers(tickers).download(period="max", auto_adjust=True, interval="1h")['Close']

[*********************100%***********************]  2 of 2 completed


In [8]:
ydata.head()

Ticker,MSTR,MSTU
Datetime,,
2023-11-10 14:30:00+00:00,50.530998,NaN
2023-11-10 15:30:00+00:00,51.019001,NaN
2023-11-10 16:30:00+00:00,50.424999,NaN
2023-11-10 17:30:00+00:00,50.698997,NaN
2023-11-10 18:30:00+00:00,51.125500,NaN


In [9]:
data = (   # put data in long form
    ydata
    .dropna()
    .stack(future_stack=True)  # Removed the level parameter since there's only one level
    .reset_index()
    .rename(columns={0: 'Close'})  # Rename the stacked column to 'Close'
    [['Datetime', 'Ticker', 'Close']]  # Note: column is 'Datetime', not 'Date'
    .sort_values(by=['Datetime', 'Ticker'])
)

data.head(5)

,Datetime,Ticker,Close
0,2024-09-18 13:30:00+00:00,MSTR,130.410004
1,2024-09-18 13:30:00+00:00,MSTU,2.415000
2,2024-09-18 14:30:00+00:00,MSTR,131.684998
3,2024-09-18 14:30:00+00:00,MSTU,2.496000
4,2024-09-18 15:30:00+00:00,MSTR,132.600006


In [ ]:
# download historical data from Yahoo Finance to csv, file name data_<tickers>_2002-07-30_D.csv
data.to_csv('data_MSTU_MSTR_2002-07-30_D.csv', index=False)

In [ ]:
# read data from csv
data = pd.read_csv('data_MSTU_MSTR_2002-07-30_D.csv')
data['Date'] = pd.to_datetime(data['Date'])
print(data.info())
print(data.head(5))

In [ ]:
# Add year-month label (end of month date)

# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Create a new column 'ym' representing the last day of each month
data['ym'] = data['Date'] + pd.offsets.MonthEnd(0)

data.head()

In [ ]:
# Create trading-day counter for each Ticker and month

# Sort data to ensure correct order
data = data.sort_values(['Ticker', 'Date']).copy()

# Count trading days within each (Ticker, ym) group
data['td'] = data.groupby(['Ticker', 'ym']).cumcount() + 1

data.head()

In [ ]:
#  Calculate daily simple returns

# Calculate percentage change of 'Close' within each ticker
data['ret'] = data.groupby('Ticker')['Close'].pct_change()

data.head()

In [ ]:
# Filter dates and keep only needed columns

# Keep data from August 2002 onwards
data = data[data['Date'] >= pd.Timestamp(2002, 8, 1)]

# Keep only useful columns
data = data[['Date', 'ym', 'Ticker', 'Close', 'ret', 'td']]

data.head()

In [ ]:
# Create start/end/month flags and conditional returns

N = 15  # number of days for start of month
M = 5   # number of days for next start of month

# Flags: 1 if condition true, 0 otherwise
data['som_flag'] = (data['td'] <= N).astype(int)
data['eom_flag'] = (data['td'] > N).astype(int)
data['nsm_flag'] = (data['td'] <= M).astype(int)

# Conditional returns (return only within flag period)
data['ret_som'] = data['ret'] * data['som_flag']
data['ret_eom'] = data['ret'] * data['eom_flag']
data['ret_nsm'] = data['ret'] * data['nsm_flag']

data.head()

In [ ]:
# Keep only relevant return columns
data_flags = data[['Date', 'ym', 'Ticker', 'ret_som', 'ret_eom', 'ret_nsm']]

data_flags.head()

In [ ]:
# Pivot to wide format (one column per Ticker)

# Create wide format table: columns for each Ticker and return type
pivoted = data_flags.pivot(index=['Date', 'ym'], columns='Ticker')

# Flatten multi-level column names (e.g. ret_som_MSTU)
pivoted.columns = [f"{col[1]}_{col[0]}" for col in pivoted.columns]

pivoted = pivoted.reset_index()

pivoted.head()

In [ ]:
# create a copy of data to be later used for the backtest analysis (8 PL streams in the backtesting)
backtest_data = pivoted.copy()

In [ ]:
# Compute relative returns and aggregate monthly

# Calculate combined and relative returns
pivoted['MSTU_ret'] = pivoted['MSTU_ret_som'] + pivoted['MSTU_ret_eom']
pivoted['MSTU_MSTR_som'] = pivoted['MSTU_ret_som'] - pivoted['MSTR_ret_som']
pivoted['MSTU_MSTR_eom'] = pivoted['MSTU_ret_eom'] - pivoted['MSTR_ret_eom']
pivoted['MSTU_MSTR_nsm'] = pivoted['MSTU_ret_nsm'] - pivoted['MSTR_ret_nsm']

# Select relevant columns
returns = pivoted[['ym', 'MSTU_ret', 'MSTU_MSTR_som', 'MSTU_MSTR_eom', 'MSTU_MSTR_nsm']]

# Aggregate monthly returns using compounding formula
monthly = returns.groupby('ym').apply(lambda x: (1 + x).prod() - 1)

monthly.head()

In [ ]:
monthly.tail(5)

In [ ]:
return_data = monthly.copy()